# Using LangChain To Create ChatBot Via Konko

## Install necessary Python packages

In [ ]:
!pip install typing-extensions==4.7.0
!pip install langchain==0.1.4
!pip install konko

## Import Libraries

In [4]:
import konko
from langchain.chat_models import ChatKonko
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import load_tools
from langchain.agents import AgentOutputParser
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.output_parsers.json import parse_json_markdown
from langchain.schema import AgentAction, AgentFinish
from langchain.agents import initialize_agent
import os

##  Set Environment Variables

In [5]:
os.environ['KONKO_API_KEY'] = 'your_KONKO_API_KEY_here'

Next, you need to set your environment variables for KONKO_API_KEY.

1. You can set environment variables for
   1. KONKO_API_KEY (Required)
   2. OPENAI_API_KEY (Optional)
2. In your current shell session, use the export command:

```shell
export KONKO_API_KEY={your_KONKO_API_KEY_here}
export OPENAI_API_KEY={your_OPENAI_API_KEY_here} #Optional
```

Alternatively, you can add the above lines directly to your shell startup script (such as .bashrc or .bash_profile for Bash shell and .zshrc for Zsh shell) to have them set automatically every time a new shell session starts.



## Set Up Your Language Model

Next, configure your language model. In this context, we'll be utilizing Konko as a provider within the langchain library, specifically using the 'llama-2-13b-chat' model.

In [6]:
llm = ChatKonko(model='meta-llama/llama-2-13b-chat', max_tokens = 1000)

## Initializing an Conversational Agent


We first need to initialize the agent. Conversational agents require several things such as conversational memory, access to tools, and an llm (which we have already initialized).

In [7]:
memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output"
)
tools = load_tools(["llm-math"], llm=llm)

##  Define the OutputParser Class

Define the OutputParser class to handle the parsing of the model's output

In [8]:
class OutputParser(AgentOutputParser):
    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> AgentAction | AgentFinish:
            return AgentFinish({"output": text}, text)

    @property
    def _type(self) -> str:
        return "conversational_chat"

# initialize output parser for agent
parser = OutputParser()

## Initialize the Agent

Next, initialize the agent with the defined parser, memory buffer, tools, and the Konko model

In [9]:
# initialize agent
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    early_stopping_method="generate",
    memory=memory,
    agent_kwargs={"output_parser": parser}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


# Set Up the Conversation Flow

Konko manages special instruction tokens, simplifying the creation of LLM-compatible prompts and streamlining your chatbot's interactions, allowing you to focus on other important aspects of your chatbot design.

In [10]:
human_msg = "{input}"

agent.agent.llm_chain.prompt.messages[2].prompt.template = human_msg

## Now we can begin asking questions...

In [11]:
output = agent("Hi, I just downloaded your photo editing app, but I'm having trouble understanding how to use the crop tool.")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
  Hello! I'd be happy to help you understand how to use the crop tool in our photo editing app. Can you tell me a little bit more about what you're having trouble with? For example, are you having trouble selecting the area you want to crop, or are you having trouble understanding how to adjust the crop size and position? The more information you can provide, the better I'll be able to assist you.

> Finished chain.


In [12]:
agent("I don't know how to adjust the size and position of the crop ")



> Entering new AgentExecutor chain...
 Sure thing! Adjusting the size and position of the crop can take a little practice, but don't worry, I'm here to help.

First, let me just give you a quick overview of the crop tool. When you select the crop tool, you'll see a rectangle with handles on the corners. The rectangle represents the area of the image that will be cropped.

To adjust the size and position of the crop, you can drag any of the handles to move the rectangle. For example, if you want to make the crop smaller, you can drag the corner handles inward. If you want to make the crop larger, you can drag the side handles outward.

Now, let's practice adjusting the crop size and position together. Let's say you want to crop this image to focus on just the flower. You can start by clicking and dragging the crop tool over the flower, and then adjusting the size and position of the crop to your liking.

As you drag the handles, you'll see the image update in real-time, so you can see

{'input': "I don't know how to adjust the size and position of the crop ",
 'chat_history': [HumanMessage(content="Hi, I just downloaded your photo editing app, but I'm having trouble understanding how to use the crop tool."),
  AIMessage(content="  Hello! I'd be happy to help you understand how to use the crop tool in our photo editing app. Can you tell me a little bit more about what you're having trouble with? For example, are you having trouble selecting the area you want to crop, or are you having trouble understanding how to adjust the crop size and position? The more information you can provide, the better I'll be able to assist you.")],
 'output': " Sure thing! Adjusting the size and position of the crop can take a little practice, but don't worry, I'm here to help.\n\nFirst, let me just give you a quick overview of the crop tool. When you select the crop tool, you'll see a rectangle with handles on the corners. The rectangle represents the area of the image that will be croppe

In [13]:
agent("Thanks that worked! Could you also guide me on how to adjust the brightness and contrast of an image obtained above?")



> Entering new AgentExecutor chain...
  Of course! Adjusting the brightness and contrast of an image can help bring out the details and make the image look more vibrant.

To adjust the brightness and contrast of an image using our photo editing app, follow these steps:

Step 1: Select the image you want to edit.

Step 2: Click on the "Adjust" button in the toolbar at the top of the screen.

Step 3: In the adjustment menu, click on the "Brightness and Contrast" option.

Step 4: Use the sliders to adjust the brightness and contrast of the image to your liking. You can use the preview image on the right-hand side of the screen to see how your adjustments will look.

Here's a quick rundown of what the brightness and contrast sliders do:

* Brightness: Adjusting the brightness will make the image lighter or darker. Turning the slider to the right will make the image brighter, while turning it to the left will make it darker.
* Contrast: Adjusting the contrast will make the image more or l

{'input': 'Thanks that worked! Could you also guide me on how to adjust the brightness and contrast of an image obtained above?',
 'chat_history': [HumanMessage(content="Hi, I just downloaded your photo editing app, but I'm having trouble understanding how to use the crop tool."),
  AIMessage(content="  Hello! I'd be happy to help you understand how to use the crop tool in our photo editing app. Can you tell me a little bit more about what you're having trouble with? For example, are you having trouble selecting the area you want to crop, or are you having trouble understanding how to adjust the crop size and position? The more information you can provide, the better I'll be able to assist you."),
  HumanMessage(content="I don't know how to adjust the size and position of the crop "),
  AIMessage(content=" Sure thing! Adjusting the size and position of the crop can take a little practice, but don't worry, I'm here to help.\n\nFirst, let me just give you a quick overview of the crop too